# 💬 TMCD – Trabalho Final
## Análise de Sentimentos em Reviews de Filmes

### 👥 Grupo Trab-grupo-30
- **Rafael Alexandre Dias Andorinha**, nº 131000  
- **Pedro Fonte Santa**, nº 105306  

---

📅 **Data de entrega:** 26 de abril  

📊 **Objetivo deste script:**
Este Notebook corresponde à Tarefa 2.4 do trabalho.

O objetivo desta fase é treinar um modelo de aprendizagem automática supervisionada para o problema de análise de sentimentos no dataset IMDB. A ideia principal consiste em utilizar modelos clássicos de classificação como baseline para comparar com abordagens mais avançadas nas etapas seguintes.

O modelo escolhido foi a Regressão Logística, utilizando uma representação dos textos através do vetor TF-IDF. Foram realizadas três experiências distintas, variando o nível de pré-processamento do texto:

- **Experiência 1**: Apenas lowercase
- **Experiência 2**: Adicionámos remoção de pontuação e stopwords
- **Experiência 3**: Adicionámos stemming

As métricas utilizadas para avaliação foram a `accuracy`, `precision`, `recall` e `F1-score`. Os resultados foram comparados visualmente, permitindo avaliar o impacto das diferentes técnicas de pré-processamento na performance do modelo.

---

# 🗂️ Dataset: IMDB Reviews

In [4]:
import pandas as pd
import numpy as np
import string
import nltk
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Downloads necessários
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pedrofs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 🔹 1. Carregar os dados

In [5]:
# Caminhos para os ficheiros
train_path = '../dataset/imdb_reviews_train.csv'
test_path = '../dataset/imdb_reviews_test.csv'

# Carregar os datasets
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Visualizar estrutura
df_train.head()

,text,label
0,This is your typical cheerful and colorful MGM...,pos
1,As a another reviewer states Hanna's War is an...,pos
2,"One of the best ""Amitabh comeback"" movies I li...",pos
3,Peter Sollett has created an endearing portrai...,pos
4,The film is not visually stunning in the conve...,pos


### 🔹 2. Função auxiliar para avaliar os resultados

In [6]:
# Avaliação
def avaliar_resultados(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label='pos')
    rec = recall_score(y_true, y_pred, pos_label='pos')
    f1 = f1_score(y_true, y_pred, pos_label='pos')
    
    print(f"Accuracy: {acc:.3f}")
    print(f"Precisão: {prec:.3f}")
    print(f"Recall: {rec:.3f}")
    print(f"F1 Score: {f1:.3f}")

    return {'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1': f1}

### ⚙️ Experiência 1 — Baseline (apenas lowercase)

In [7]:
# Pré-processamento simples
df_train['text_exp1'] = df_train['text'].str.lower()
df_test['text_exp1'] = df_test['text'].str.lower()

# TF-IDF
vectorizer1 = TfidfVectorizer()
X_train1 = vectorizer1.fit_transform(df_train['text_exp1'])
X_test1 = vectorizer1.transform(df_test['text_exp1'])

# Modelo
modelo1 = LogisticRegression(max_iter=1000)
modelo1.fit(X_train1, df_train['label'])
y_pred1 = modelo1.predict(X_test1)

# Avaliação
print("📊 Resultados — Experiência 1 (baseline, apenas lowercase)")
resultados_exp1 = avaliar_resultados(df_test['label'], y_pred1)

📊 Resultados — Experiência 1 (baseline, apenas lowercase)
Accuracy: 0.882
Precisão: 0.884
Recall: 0.879
F1 Score: 0.882


### ⚙️ Experiência 2 — Com stopwords e pontuação removida

In [8]:
# Pré-processamento com stopwords e pontuação removida
stop_words = set(stopwords.words('english'))
pontuacao = set(string.punctuation)

def preprocessar_texto_limpo(texto):
    texto = texto.lower()
    tokens = wordpunct_tokenize(texto)
    tokens_filtrados = [t for t in tokens if t not in pontuacao and t not in stop_words]
    return ' '.join(tokens_filtrados)


In [9]:
# Pré-processar texto
df_train['text_exp2'] = df_train['text'].apply(preprocessar_texto_limpo)
df_test['text_exp2'] = df_test['text'].apply(preprocessar_texto_limpo)

# TF-IDF
vectorizer2 = TfidfVectorizer()
X_train2 = vectorizer2.fit_transform(df_train['text_exp2'])
X_test2 = vectorizer2.transform(df_test['text_exp2'])

# Modelo
modelo2 = LogisticRegression(max_iter=1000)
modelo2.fit(X_train2, df_train['label'])
y_pred2 = modelo2.predict(X_test2)

# Avaliação
print("📊 Resultados — Experiência 2 (com stopwords e pontuação removida)")
resultados_exp2 = avaliar_resultados(df_test['label'], y_pred2)

📊 Resultados — Experiência 2 (com stopwords e pontuação removida)
Accuracy: 0.882
Precisão: 0.882
Recall: 0.881
F1 Score: 0.882


### ⚙️ Experiência 3 — Com stemming

In [10]:
# Pré-processamento com stemming
stemmer = PorterStemmer()

def preprocessar_texto_stemming(texto):
    texto = texto.lower()
    tokens = wordpunct_tokenize(texto)
    tokens = [t for t in tokens if t not in pontuacao and t not in stop_words]
    stems = [stemmer.stem(t) for t in tokens]
    return ' '.join(stems)


In [11]:
# Pré-processar texto com stemming
df_train['text_exp3'] = df_train['text'].apply(preprocessar_texto_stemming)
df_test['text_exp3'] = df_test['text'].apply(preprocessar_texto_stemming)

# TF-IDF
vectorizer3 = TfidfVectorizer()
X_train3 = vectorizer3.fit_transform(df_train['text_exp3'])
X_test3 = vectorizer3.transform(df_test['text_exp3'])

# Modelo
modelo3 = LogisticRegression(max_iter=1000)
modelo3.fit(X_train3, df_train['label'])
y_pred3 = modelo3.predict(X_test3)

# Avaliação
print("📊 Resultados — Experiência 3 (com stemming)")
resultados_exp3 = avaliar_resultados(df_test['label'], y_pred3)


📊 Resultados — Experiência 3 (com stemming)
Accuracy: 0.880
Precisão: 0.880
Recall: 0.879
F1 Score: 0.879


### 📈 Comparação final das experiências

In [ ]:
# Preparar DataFrame com resultados
valores = [
    list(resultados_exp1.values()),
    list(resultados_exp2.values()),
    list(resultados_exp3.values())
]

experiencias = ['Exp. 1\nBaseline', 'Exp. 2\n+ Stopwords', 'Exp. 3\n+ Stemming']
metricas = ['Accuracy', 'Precisão', 'Recall', 'F1 Score']

df_resultados = pd.DataFrame(valores, index=experiencias, columns=metricas)

# Gráfico
plt.figure(figsize=(10, 6))
df_resultados.plot(kind='bar', figsize=(10, 6))
plt.title('Comparação de desempenho entre experiências de ML')
plt.ylabel('Valor da Métrica')
plt.ylim(0.5, 0.9)
plt.xticks(rotation=0)
plt.legend(loc='lower right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### 📊 Resultados das Experiências com Modelos de Aprendizagem Automática

| Experiência            | Accuracy | Precision | Recall | F1 Score | Observações                                 |
|------------------------|----------|----------|--------|----------|---------------------------------------------|
| Exp. 1 - Baseline      | 0.882    | 0.884    | 0.879  | 0.882    | Apenas lowercase                            |
| Exp. 2 - + Stopwords   | 0.882    | 0.882    | 0.881  | 0.882    | Remoção de stopwords e pontuação            |
| Exp. 3 - + Stemming    | 0.880    | 0.880    | 0.879  | 0.879    | Pré-processamento com stemming              |
